# News Preprocessing Using Ckiplab NLP Package

多篇新聞熱門斷詞關鍵字統計新聞摘要-整理並存檔

# All-in-one

一鍵完工

In [ ]:
%%time
import pandas as pd
import numpy
from collections import Counter
from ckip_transformers.nlp import CkipWordSegmenter, CkipPosTagger, CkipNerChunker

df = pd.read_csv('cna_category_news.csv', sep='|')

# ckiplab word segment (中研院斷詞)
# Initialize drivers
# It takes time to download ckiplab models

# default參數是model="bert-base"
# ws = CkipWordSegmenter() 
# pos = CkipPosTagger()
# ner = CkipNerChunker()

# model="albert-tiny" 模型小，斷詞速度比較快，犧牲一些精確度
ws = CkipWordSegmenter(model="albert-tiny") 
pos = CkipPosTagger(model="albert-tiny")
ner = CkipNerChunker(model="albert-tiny")


## Word Segmentation
tokens = ws(df.content)

## POS
tokens_pos = pos(tokens)

## word pos pair 詞性關鍵字
word_pos_pair = [list(zip(article_word, article_pos)) for article_word, article_pos in zip(tokens, tokens_pos)]

## NER命名實體辨識
entity_list = ner(df.content)

# Remove stop words and filter using POS tag (tokens_v2)
#with open('stops_chinese_traditional.txt', 'r', encoding='utf8') as f:
#    stops = f.read().split('\n')

# 過濾條件:兩個字以上 特定的詞性
# allowPOS 過濾條件: 特定的詞性
allowPOS = ['Na', 'Nb', 'Nc']

tokens_v2 = []
for wp in word_pos_pair:
    tokens_v2.append([w for w, p in wp if (len(w) >= 2) and p in allowPOS])

# Insert tokens into dataframe (新增斷詞資料欄位)
df['tokens'] = tokens
df['tokens_v2'] = tokens_v2
df['entities'] = entity_list
df['token_pos'] = word_pos_pair

# Calculate word count (frequency) 計算字頻(次數)


def word_frequency(wp_pair):
    filtered_words = []
    for word, pos in wp_pair:
        if (pos in allowPOS) & (len(word) >= 2):
            filtered_words.append(word)
        #print('%s %s' % (word, pos))
    counter = Counter(filtered_words)
    return counter.most_common(200)


keyfreqs = []
for wp in word_pos_pair:
    topwords = word_frequency(wp)
    keyfreqs.append(topwords)

df['top_key_freq'] = keyfreqs

# Abstract (summary) and sentimental score(摘要與情緒分數)
summary = []
sentiment = []
for text in df.content:
    summary.append("暫無")
    sentiment.append("暫無")

df['summary'] = summary
df['sentiment'] = sentiment

# Rearrange the colmun order for readability
df = df[[
    'item_id', 'date','category', 'title', 'content', 'sentiment', 'summary',
    'top_key_freq', 'tokens', 'tokens_v2', 'entities', 'token_pos', 'link',
    'photo_link'
]]

# Save data to disk
df.to_csv('cna_news_preprocessed.csv', sep='|', index=False)

## Read it out 讀出看看
#df = pd.read_csv('cna_dataset_preprocessed.csv', sep='|')
#df.head(1)

print("Tokenize OK!")

Inference: 100%|██████████| 1/1 [00:00<00:00,  4.33it/s]

Tokenize OK!
CPU times: total: 24.2 s
Wall time: 8.55 s


In [2]:
## Read it out 讀出看看
df = pd.read_csv('cna_news_preprocessed.csv', sep='|')
df.head(1)


,item_id,date,category,title,content,sentiment,summary,top_key_freq,tokens,tokens_v2,entities,token_pos,link,photo_link
0,aipl_20250304_1,2025-03-04,政治,梵蒂岡圖書館長率團參觀台灣使館 力薦值得訪台,梵蒂岡宗座圖書館總館長札尼今天率領數十位北義的社會菁英教友拜會駐教廷大使館。曾訪台的札尼致詞...,暫無,暫無,"[('台灣', 20), ('教友', 7), ('札尼', 4), ('布雷西亞', 4)...","['梵蒂岡', '宗座', '圖書館', '總', '館長', '札尼', '今天', '率...","['梵蒂岡', '宗座', '圖書館', '館長', '札尼', '北義', '社會', '...","[NerToken(word='梵蒂岡', ner='GPE', idx=(0, 3)), ...","[('梵蒂岡', 'Nc'), ('宗座', 'Na'), ('圖書館', 'Nc'), (...",/news/aipl/202503040022.aspx,https://imgcdn.cna.com.tw/www/WebPhotos/200/20...


# Demonstration Step by Step 

# Read data from file

In [3]:
import pandas as pd
import numpy

In [4]:
df = pd.read_csv('cna_category_news.csv', sep='|')

In [5]:
df.shape

(12, 7)

In [6]:
df.head()

,item_id,date,category,title,content,link,photo_link
0,aipl_20250304_1,2025-03-04,政治,梵蒂岡圖書館長率團參觀台灣使館 力薦值得訪台,梵蒂岡宗座圖書館總館長札尼今天率領數十位北義的社會菁英教友拜會駐教廷大使館。曾訪台的札尼致詞...,/news/aipl/202503040022.aspx,https://imgcdn.cna.com.tw/www/WebPhotos/200/20...
1,aipl_20250304_2,2025-03-04,政治,力挺友邦瓜地馬拉 台灣採購咖啡增加5成創新高,由外交部中美洲經貿辦事處籌組的台灣咖啡考察團，力挺友邦瓜地馬拉，預計採購38個貨櫃、72萬公...,/news/aipl/202503040019.aspx,https://imgcdn.cna.com.tw/www/WebPhotos/200/20...
2,aipl_20250303_3,2025-03-03,政治,民團告發傅崐萁涉外患與強制罪 高檢署：依法辦理,經民連與台灣東社今天分別告發國民黨立委傅崐萁涉犯外患罪、反滲透法與強制罪；高檢署表示，已收到...,/news/aipl/202503030336.aspx,https://imgcdn.cna.com.tw/www/WebPhotos/200/20...
3,aipl_20250303_4,2025-03-03,政治,民眾黨公聽會 學者：不認同超徵後普發現金,台灣去年度稅收高於預算，國民黨主張普發現金新台幣1萬元。民眾黨團今天舉行公聽會，會中學者認為...,/news/aipl/202503030327.aspx,NaN
4,ait_20250303_1,2025-03-03,科技,印度推動碳市場 計劃2026年中上路,印度電力部長卡塔爾表示，印度正積極準備在2026年底前推出備受期待的碳市場。這項政策預計發揮...,https://netzero.cna.com.tw/news/202503030214/,https://imgcdn.cna.com.tw/www/WebPhotos/200/20...


In [7]:
df.content[0:5]

0    梵蒂岡宗座圖書館總館長札尼今天率領數十位北義的社會菁英教友拜會駐教廷大使館。曾訪台的札尼致詞...
1    由外交部中美洲經貿辦事處籌組的台灣咖啡考察團，力挺友邦瓜地馬拉，預計採購38個貨櫃、72萬公...
2    經民連與台灣東社今天分別告發國民黨立委傅崐萁涉犯外患罪、反滲透法與強制罪；高檢署表示，已收到...
3    台灣去年度稅收高於預算，國民黨主張普發現金新台幣1萬元。民眾黨團今天舉行公聽會，會中學者認為...
4    印度電力部長卡塔爾表示，印度正積極準備在2026年底前推出備受期待的碳市場。這項政策預計發揮...
Name: content, dtype: object

In [8]:
df['content'][0:5]

0    梵蒂岡宗座圖書館總館長札尼今天率領數十位北義的社會菁英教友拜會駐教廷大使館。曾訪台的札尼致詞...
1    由外交部中美洲經貿辦事處籌組的台灣咖啡考察團，力挺友邦瓜地馬拉，預計採購38個貨櫃、72萬公...
2    經民連與台灣東社今天分別告發國民黨立委傅崐萁涉犯外患罪、反滲透法與強制罪；高檢署表示，已收到...
3    台灣去年度稅收高於預算，國民黨主張普發現金新台幣1萬元。民眾黨團今天舉行公聽會，會中學者認為...
4    印度電力部長卡塔爾表示，印度正積極準備在2026年底前推出備受期待的碳市場。這項政策預計發揮...
Name: content, dtype: object

# Ckiplab word segment (中研院斷詞)

# Load tokenization models


In [9]:
from ckip_transformers.nlp import CkipWordSegmenter, CkipPosTagger, CkipNerChunker

In [10]:
# model="albert-tiny" 模型小，斷詞速度比較快，犧牲一些精確度
ws = CkipWordSegmenter(model="albert-tiny") 
pos = CkipPosTagger(model="albert-tiny")
ner = CkipNerChunker(model="albert-tiny")

## Word Segmentation

In [11]:
%%time
# It takes time.
tokens = ws(df.content)

Inference: 100%|██████████| 1/1 [00:00<00:00,  4.05it/s]

CPU times: total: 3.09 s
Wall time: 264 ms


In [12]:
len(tokens)

12

In [13]:
len(tokens[0])

492

In [14]:
tokens[0]

['梵蒂岡',
 '宗座',
 '圖書館',
 '總',
 '館長',
 '札尼',
 '今天',
 '率領',
 '數十',
 '位',
 '北義',
 '的',
 '社會',
 '菁英',
 '教友',
 '拜會',
 '駐',
 '教廷',
 '大使館',
 '。',
 '曾',
 '訪',
 '台',
 '的',
 '札尼',
 '致詞',
 '時',
 '向',
 '教友',
 '力',
 '推',
 '「',
 '台灣',
 '是',
 '個',
 '卓越',
 '國家',
 '，',
 '非常',
 '值得',
 '前往',
 '」',
 '，',
 '多',
 '位',
 '教友',
 '對',
 '台灣',
 '高度',
 '好奇',
 '，',
 '熱情',
 '表示',
 '「',
 '下',
 '次',
 '旅遊',
 '地點',
 '將',
 '鎖定',
 '台灣',
 '」',
 '。',
 '札尼',
 '（',
 'Angelo Vincenzo',
 ' Zani',
 '）',
 '於',
 '2023年',
 '曾',
 '訪',
 '台',
 '參加',
 '故宮',
 '「',
 '梵蒂岡',
 '宗座',
 '圖書館',
 '珍藏',
 '」',
 '暨',
 '「',
 '明清',
 '宮廷',
 '藏書',
 '」',
 '特展',
 '，',
 '對',
 '台灣',
 '留下',
 '深刻',
 '印象',
 '。',
 '他',
 '今天',
 '率領',
 '北義',
 '家鄉',
 '布雷西亞',
 '（',
 'Brescia',
 '）',
 '約',
 '40',
 '位',
 '教友',
 '到',
 '羅馬',
 '朝聖',
 '，',
 '特別',
 '安',
 '排教友',
 '參觀',
 '台灣',
 '使館',
 '。',
 '這',
 '群',
 '教友',
 '皆',
 '為',
 '布雷西亞',
 '扶輪社',
 '成員',
 '，',
 '職業',
 '包括',
 '中',
 '小企業家',
 '、',
 '大學',
 '教授',
 '、',
 '律師',
 '和',
 '高階',
 '憲警',
 '等',
 '。',
 '札尼',
 '致詞',
 '時',
 '盛讚',
 '，',

In [15]:
tokens[0][0]

'梵蒂岡'

## POS

In [16]:
%%time
tokens_pos = pos(tokens)

Inference: 100%|██████████| 2/2 [00:00<00:00,  2.45it/s]

CPU times: total: 12.2 s
Wall time: 844 ms


In [17]:
len(tokens_pos)

12

In [18]:
len(tokens_pos[0])

492

In [19]:
tokens_pos[0]

['Nc',
 'Na',
 'Nc',
 'Nc',
 'Na',
 'Nb',
 'Nd',
 'VF',
 'Neu',
 'Nf',
 'Nc',
 'DE',
 'Na',
 'Na',
 'Na',
 'VC',
 'VCL',
 'Nc',
 'Nc',
 'PERIODCATEGORY',
 'D',
 'VC',
 'Nc',
 'DE',
 'Nb',
 'VA',
 'Ng',
 'P',
 'Na',
 'VC',
 'VC',
 'PARENTHESISCATEGORY',
 'Nc',
 'SHI',
 'Nf',
 'VH',
 'Na',
 'COMMACATEGORY',
 'Dfa',
 'VH',
 'VCL',
 'PARENTHESISCATEGORY',
 'COMMACATEGORY',
 'Neqa',
 'Nf',
 'Na',
 'P',
 'Nc',
 'A',
 'VH',
 'COMMACATEGORY',
 'VH',
 'VE',
 'PARENTHESISCATEGORY',
 'Nes',
 'Nf',
 'Nv',
 'Na',
 'D',
 'VC',
 'Nc',
 'PARENTHESISCATEGORY',
 'PERIODCATEGORY',
 'Nb',
 'PARENTHESISCATEGORY',
 'FW',
 'FW',
 'PARENTHESISCATEGORY',
 'P',
 'Neu',
 'D',
 'VC',
 'Nc',
 'VC',
 'Nc',
 'PARENTHESISCATEGORY',
 'Nb',
 'Nb',
 'Nc',
 'VC',
 'PARENTHESISCATEGORY',
 'Caa',
 'PARENTHESISCATEGORY',
 'Nd',
 'Nc',
 'Na',
 'PARENTHESISCATEGORY',
 'Na',
 'COMMACATEGORY',
 'P',
 'Nc',
 'VC',
 'VH',
 'Na',
 'PERIODCATEGORY',
 'Nh',
 'Nd',
 'VF',
 'Nc',
 'Nc',
 'Nc',
 'PARENTHESISCATEGORY',
 'FW',
 'PARENTHE

## word pos pair 詞性關鍵字

tokens and tokens_pos are two dimentional list with serveral articles. 

In [ ]:
for article_word, article_pos in zip(tokens,tokens_pos):
    print(article_word)
    print(article_pos)

['梵蒂岡', '宗座', '圖書館', '總', '館長', '札尼', '今天', '率領', '數十', '位', '北義', '的', '社會', '菁英', '教友', '拜會', '駐', '教廷', '大使館', '。', '曾', '訪', '台', '的', '札尼', '致詞', '時', '向', '教友', '力', '推', '「', '台灣', '是', '個', '卓越', '國家', '，', '非常', '值得', '前往', '」', '，', '多', '位', '教友', '對', '台灣', '高度', '好奇', '，', '熱情', '表示', '「', '下', '次', '旅遊', '地點', '將', '鎖定', '台灣', '」', '。', '札尼', '（', 'Angelo Vincenzo', ' Zani', '）', '於', '2023年', '曾', '訪', '台', '參加', '故宮', '「', '梵蒂岡', '宗座', '圖書館', '珍藏', '」', '暨', '「', '明清', '宮廷', '藏書', '」', '特展', '，', '對', '台灣', '留下', '深刻', '印象', '。', '他', '今天', '率領', '北義', '家鄉', '布雷西亞', '（', 'Brescia', '）', '約', '40', '位', '教友', '到', '羅馬', '朝聖', '，', '特別', '安', '排教友', '參觀', '台灣', '使館', '。', '這', '群', '教友', '皆', '為', '布雷西亞', '扶輪社', '成員', '，', '職業', '包括', '中', '小企業家', '、', '大學', '教授', '、', '律師', '和', '高階', '憲警', '等', '。', '札尼', '致詞', '時', '盛讚', '，', '台灣', '長期', '與', '教廷', '攜手', '推動', '人道', '援助', '，', '包括', '協助', '他', '在', '非洲', '剛果', '推動', '的', '教育', '與', '醫療', '工作', '，', '在COVID-19', '（', '2

In [ ]:
word_pos_pair = [list(zip(article_word,article_pos)) for article_word, article_pos in zip(tokens, tokens_pos)]

In [66]:
len(word_pos_pair)

12

In [67]:
word_pos_pair

[[('梵蒂岡', 'Nc'),
  ('宗座', 'Na'),
  ('圖書館', 'Nc'),
  ('總', 'Nc'),
  ('館長', 'Na'),
  ('札尼', 'Nb'),
  ('今天', 'Nd'),
  ('率領', 'VF'),
  ('數十', 'Neu'),
  ('位', 'Nf'),
  ('北義', 'Nc'),
  ('的', 'DE'),
  ('社會', 'Na'),
  ('菁英', 'Na'),
  ('教友', 'Na'),
  ('拜會', 'VC'),
  ('駐', 'VCL'),
  ('教廷', 'Nc'),
  ('大使館', 'Nc'),
  ('。', 'PERIODCATEGORY'),
  ('曾', 'D'),
  ('訪', 'VC'),
  ('台', 'Nc'),
  ('的', 'DE'),
  ('札尼', 'Nb'),
  ('致詞', 'VA'),
  ('時', 'Ng'),
  ('向', 'P'),
  ('教友', 'Na'),
  ('力', 'VC'),
  ('推', 'VC'),
  ('「', 'PARENTHESISCATEGORY'),
  ('台灣', 'Nc'),
  ('是', 'SHI'),
  ('個', 'Nf'),
  ('卓越', 'VH'),
  ('國家', 'Na'),
  ('，', 'COMMACATEGORY'),
  ('非常', 'Dfa'),
  ('值得', 'VH'),
  ('前往', 'VCL'),
  ('」', 'PARENTHESISCATEGORY'),
  ('，', 'COMMACATEGORY'),
  ('多', 'Neqa'),
  ('位', 'Nf'),
  ('教友', 'Na'),
  ('對', 'P'),
  ('台灣', 'Nc'),
  ('高度', 'A'),
  ('好奇', 'VH'),
  ('，', 'COMMACATEGORY'),
  ('熱情', 'VH'),
  ('表示', 'VE'),
  ('「', 'PARENTHESISCATEGORY'),
  ('下', 'Nes'),
  ('次', 'Nf'),
  ('旅遊', 'Nv'),
  ('地點', 'N

## How does zip work?

There are two zip. How does it work?

list(zip(w,p)) for w, p in zip(tokens, tokens_pos)

In [12]:
a=[1,2,3]
b=['a','b','c']
ab=zip(a,b)
list(ab)

[(1, 'a'), (2, 'b'), (3, 'c')]

In [5]:
# list(zip(w,p)) for w, p in zip(tokens, tokens_pos)
a=[[1,2],[3,4],[5,6]]
b=[['a','b'],['c','d'],['e','f']]
ab=zip(a,b)
list(ab)


[([1, 2], ['a', 'b']), ([3, 4], ['c', 'd']), ([5, 6], ['e', 'f'])]

In [8]:
for item1, item2 in zip(a, b):
    print(item1,'with', item2)

[1, 2] with ['a', 'b']
[3, 4] with ['c', 'd']
[5, 6] with ['e', 'f']


In [11]:
for item1, item2 in zip(a, b):
    print(item1,'with', item2)
    print(list(zip(item1, item2)))
    print('---')

[1, 2] with ['a', 'b']
[(1, 'a'), (2, 'b')]
---
[3, 4] with ['c', 'd']
[(3, 'c'), (4, 'd')]
---
[5, 6] with ['e', 'f']
[(5, 'e'), (6, 'f')]
---


## NER命名實體辨識

In [24]:
%%time
entity_list = ner(df.content)

Inference: 100%|██████████| 1/1 [00:00<00:00,  4.49it/s]

CPU times: total: 2.94 s
Wall time: 252 ms


In [25]:
entity_list[0]

[NerToken(word='梵蒂岡', ner='GPE', idx=(0, 3)),
 NerToken(word='札尼', ner='PERSON', idx=(11, 13)),
 NerToken(word='北義', ner='GPE', idx=(20, 22)),
 NerToken(word='札尼', ner='PERSON', idx=(42, 44)),
 NerToken(word='台灣', ner='GPE', idx=(53, 55)),
 NerToken(word='台灣', ner='GPE', idx=(75, 77)),
 NerToken(word='台灣', ner='GPE', idx=(96, 98)),
 NerToken(word='札尼', ner='ORG', idx=(100, 102)),
 NerToken(word='2023年', ner='DATE', idx=(125, 130)),
 NerToken(word='台', ner='GPE', idx=(132, 133)),
 NerToken(word='台灣', ner='GPE', idx=(162, 164)),
 NerToken(word='今天', ner='DATE', idx=(172, 174)),
 NerToken(word='北義', ner='GPE', idx=(176, 178)),
 NerToken(word='布雷西亞', ner='ORG', idx=(180, 184)),
 NerToken(word='4', ner='CARDINAL', idx=(194, 195)),
 NerToken(word='羅馬', ner='GPE', idx=(200, 202)),
 NerToken(word='台灣', ner='GPE', idx=(213, 215)),
 NerToken(word='布雷西亞扶輪社', ner='ORG', idx=(224, 231)),
 NerToken(word='札尼', ner='PERSON', idx=(258, 260)),
 NerToken(word='台灣', ner='GPE', idx=(266, 268)),
 NerToken(w

# Remove stop words and filter using POS (tokens_v2)

去除停用詞並依據詞性過濾

In [26]:
# with open('stops_chinese_traditional.txt', 'r', encoding='utf8') as f:
#     stops = f.read().split('\n') 

In [27]:
# 過濾條件:兩個字以上與特定的詞性
allowPOS=['Na','Nb','Nc']

tokens_v2 =[]
for wp in word_pos_pair:
    tokens_v2.append([w for w,p in wp if (len(w) >= 2) and p in allowPOS])

In [28]:
tokens_v2[0]

['梵蒂岡',
 '宗座',
 '圖書館',
 '館長',
 '札尼',
 '北義',
 '社會',
 '菁英',
 '教友',
 '教廷',
 '大使館',
 '札尼',
 '教友',
 '台灣',
 '國家',
 '教友',
 '台灣',
 '地點',
 '台灣',
 '札尼',
 '故宮',
 '梵蒂岡',
 '宗座',
 '圖書館',
 '宮廷',
 '藏書',
 '特展',
 '台灣',
 '印象',
 '北義',
 '家鄉',
 '布雷西亞',
 '教友',
 '羅馬',
 '台灣',
 '使館',
 '教友',
 '布雷西亞',
 '扶輪社',
 '成員',
 '職業',
 '大學',
 '教授',
 '律師',
 '憲警',
 '札尼',
 '台灣',
 '教廷',
 '人道',
 '援助',
 '非洲',
 '剛果',
 '教育',
 '工作',
 '冠狀',
 '病毒',
 '疾病',
 '疫情',
 '期間',
 '台灣',
 '布雷西亞',
 '醫院',
 '呼吸器',
 '家鄉教友',
 '台灣',
 '機會',
 '教廷',
 '大使',
 '李世明',
 '台灣',
 '時事哏',
 '眾人鼓掌大笑',
 '美國',
 '總統',
 '川普',
 '烏克蘭',
 '總統',
 '澤倫斯基',
 '白宮',
 '流行語',
 '國際社會',
 '台灣',
 '領土',
 '面積',
 '台灣',
 '國際',
 '重要性',
 '方面',
 '李世明',
 '台灣',
 '地緣',
 '戰略',
 '重要性',
 '台灣',
 '海峽',
 '全球',
 '貨運',
 '管道',
 '台灣',
 '半導體',
 '產業',
 '世界',
 '關鍵',
 '地位',
 '晶片',
 '製程',
 '全球',
 '市占率',
 '李世明',
 '教宗方',
 '濟各',
 '台灣',
 '國家',
 '中國',
 '威權',
 '人民',
 '希望',
 '中國',
 '人民',
 '自由',
 '正義',
 '台灣',
 '希望',
 '角色',
 '李世明',
 '北義菁英',
 '教友',
 '台灣',
 '羅馬',
 '米蘭',
 '台灣',
 '班機',
 '布雷西亞',
 '扶輪社',
 '幹部',
 '反應',
 '台灣',
 

# Insert tokens into dataframe (新增斷詞資料欄位)

In [29]:
df['tokens'] = tokens

In [30]:
df['tokens_v2'] = tokens_v2

In [31]:
df['entities'] = entity_list

In [32]:
df['token_pos'] = word_pos_pair

In [33]:
df.head(1)

,item_id,date,category,title,content,link,photo_link,tokens,tokens_v2,entities,token_pos
0,aipl_20250304_1,2025-03-04,政治,梵蒂岡圖書館長率團參觀台灣使館 力薦值得訪台,梵蒂岡宗座圖書館總館長札尼今天率領數十位北義的社會菁英教友拜會駐教廷大使館。曾訪台的札尼致詞...,/news/aipl/202503040022.aspx,https://imgcdn.cna.com.tw/www/WebPhotos/200/20...,"[梵蒂岡, 宗座, 圖書館, 總, 館長, 札尼, 今天, 率領, 數十, 位, 北義, 的...","[梵蒂岡, 宗座, 圖書館, 館長, 札尼, 北義, 社會, 菁英, 教友, 教廷, 大使館...","[(梵蒂岡, GPE, (0, 3)), (札尼, PERSON, (11, 13)), (...","[(梵蒂岡, Nc), (宗座, Na), (圖書館, Nc), (總, Nc), (館長,..."


# Calculate word count (frequency) 計算字頻(次數)

In [34]:
# allowPOS過濾條件:兩個字以上與特定的詞性
# allowPOS=['Na','Nb','Nc','VA','VAC','VB','VC']
allowPOS=['Na','Nb','Nc']

from collections import Counter
def word_frequency( wp_pair ):
    filtered_words =[]
    for word, pos in wp_pair:
        if (pos in allowPOS) & (len(word) >= 2):
            filtered_words.append(word)
        #print('%s %s' % (word, pos))
    counter = Counter( filtered_words)
    return counter.most_common(20)

In [35]:
word_pos_pair[0]

[('梵蒂岡', 'Nc'),
 ('宗座', 'Na'),
 ('圖書館', 'Nc'),
 ('總', 'Nc'),
 ('館長', 'Na'),
 ('札尼', 'Nb'),
 ('今天', 'Nd'),
 ('率領', 'VF'),
 ('數十', 'Neu'),
 ('位', 'Nf'),
 ('北義', 'Nc'),
 ('的', 'DE'),
 ('社會', 'Na'),
 ('菁英', 'Na'),
 ('教友', 'Na'),
 ('拜會', 'VC'),
 ('駐', 'VCL'),
 ('教廷', 'Nc'),
 ('大使館', 'Nc'),
 ('。', 'PERIODCATEGORY'),
 ('曾', 'D'),
 ('訪', 'VC'),
 ('台', 'Nc'),
 ('的', 'DE'),
 ('札尼', 'Nb'),
 ('致詞', 'VA'),
 ('時', 'Ng'),
 ('向', 'P'),
 ('教友', 'Na'),
 ('力', 'VC'),
 ('推', 'VC'),
 ('「', 'PARENTHESISCATEGORY'),
 ('台灣', 'Nc'),
 ('是', 'SHI'),
 ('個', 'Nf'),
 ('卓越', 'VH'),
 ('國家', 'Na'),
 ('，', 'COMMACATEGORY'),
 ('非常', 'Dfa'),
 ('值得', 'VH'),
 ('前往', 'VCL'),
 ('」', 'PARENTHESISCATEGORY'),
 ('，', 'COMMACATEGORY'),
 ('多', 'Neqa'),
 ('位', 'Nf'),
 ('教友', 'Na'),
 ('對', 'P'),
 ('台灣', 'Nc'),
 ('高度', 'A'),
 ('好奇', 'VH'),
 ('，', 'COMMACATEGORY'),
 ('熱情', 'VH'),
 ('表示', 'VE'),
 ('「', 'PARENTHESISCATEGORY'),
 ('下', 'Nes'),
 ('次', 'Nf'),
 ('旅遊', 'Nv'),
 ('地點', 'Na'),
 ('將', 'D'),
 ('鎖定', 'VC'),
 ('台灣', 'Nc'),
 ('」', 'PA

In [36]:
word_frequency(word_pos_pair[0])

[('台灣', 20),
 ('教友', 7),
 ('札尼', 4),
 ('布雷西亞', 4),
 ('李世明', 4),
 ('教廷', 3),
 ('扶輪社', 3),
 ('梵蒂岡', 2),
 ('宗座', 2),
 ('圖書館', 2),
 ('北義', 2),
 ('國家', 2),
 ('羅馬', 2),
 ('使館', 2),
 ('總統', 2),
 ('重要性', 2),
 ('全球', 2),
 ('中國', 2),
 ('人民', 2),
 ('希望', 2)]

In [37]:
word_frequency(word_pos_pair[1])

[('咖啡', 28),
 ('台灣', 18),
 ('瓜地馬拉', 9),
 ('瓜國', 6),
 ('中美洲', 5),
 ('考察團', 5),
 ('經貿', 4),
 ('業者', 4),
 ('價格', 4),
 ('外交部', 3),
 ('辦事處', 3),
 ('貨櫃', 3),
 ('產區', 3),
 ('瓜地', 3),
 ('馬拉', 3),
 ('氣候', 3),
 ('徐韶慧', 2),
 ('咖啡豆', 2),
 ('進口商', 2),
 ('全球', 2)]

In [38]:
word_frequency(word_pos_pair[2])

[('傅崐萁', 6),
 ('台灣', 5),
 ('中國', 3),
 ('官員', 3),
 ('國民黨', 2),
 ('行政院', 2),
 ('香港', 2),
 ('政府', 2),
 ('花蓮', 2),
 ('葉春蓮', 2),
 ('東社', 1),
 ('立委', 1),
 ('外患罪', 1),
 ('滲透法', 1),
 ('強制罪', 1),
 ('高檢署', 1),
 ('告發狀', 1),
 ('經濟', 1),
 ('民主', 1),
 ('連合智庫', 1)]

# 逐篇計算詞頻

In [39]:
%%time
keyfreqs =[]
for wp in word_pos_pair:
    topwords = word_frequency(wp)
    keyfreqs.append(topwords)  

CPU times: total: 0 ns
Wall time: 997 μs


In [40]:
keyfreqs[0:2]

[[('台灣', 20),
  ('教友', 7),
  ('札尼', 4),
  ('布雷西亞', 4),
  ('李世明', 4),
  ('教廷', 3),
  ('扶輪社', 3),
  ('梵蒂岡', 2),
  ('宗座', 2),
  ('圖書館', 2),
  ('北義', 2),
  ('國家', 2),
  ('羅馬', 2),
  ('使館', 2),
  ('總統', 2),
  ('重要性', 2),
  ('全球', 2),
  ('中國', 2),
  ('人民', 2),
  ('希望', 2)],
 [('咖啡', 28),
  ('台灣', 18),
  ('瓜地馬拉', 9),
  ('瓜國', 6),
  ('中美洲', 5),
  ('考察團', 5),
  ('經貿', 4),
  ('業者', 4),
  ('價格', 4),
  ('外交部', 3),
  ('辦事處', 3),
  ('貨櫃', 3),
  ('產區', 3),
  ('瓜地', 3),
  ('馬拉', 3),
  ('氣候', 3),
  ('徐韶慧', 2),
  ('咖啡豆', 2),
  ('進口商', 2),
  ('全球', 2)]]

In [41]:
df['top_key_freq'] = keyfreqs

In [42]:
df.head(2)

,item_id,date,category,title,content,link,photo_link,tokens,tokens_v2,entities,token_pos,top_key_freq
0,aipl_20250304_1,2025-03-04,政治,梵蒂岡圖書館長率團參觀台灣使館 力薦值得訪台,梵蒂岡宗座圖書館總館長札尼今天率領數十位北義的社會菁英教友拜會駐教廷大使館。曾訪台的札尼致詞...,/news/aipl/202503040022.aspx,https://imgcdn.cna.com.tw/www/WebPhotos/200/20...,"[梵蒂岡, 宗座, 圖書館, 總, 館長, 札尼, 今天, 率領, 數十, 位, 北義, 的...","[梵蒂岡, 宗座, 圖書館, 館長, 札尼, 北義, 社會, 菁英, 教友, 教廷, 大使館...","[(梵蒂岡, GPE, (0, 3)), (札尼, PERSON, (11, 13)), (...","[(梵蒂岡, Nc), (宗座, Na), (圖書館, Nc), (總, Nc), (館長,...","[(台灣, 20), (教友, 7), (札尼, 4), (布雷西亞, 4), (李世明, ..."
1,aipl_20250304_2,2025-03-04,政治,力挺友邦瓜地馬拉 台灣採購咖啡增加5成創新高,由外交部中美洲經貿辦事處籌組的台灣咖啡考察團，力挺友邦瓜地馬拉，預計採購38個貨櫃、72萬公...,/news/aipl/202503040019.aspx,https://imgcdn.cna.com.tw/www/WebPhotos/200/20...,"[由, 外交部, 中美洲, 經貿, 辦事處, 籌組, 的, 台灣, 咖啡, 考察團, ，, ...","[外交部, 中美洲, 經貿, 辦事處, 台灣, 咖啡, 考察團, 友邦, 瓜地馬拉, 貨櫃,...","[(外交部中美洲經貿辦事處, ORG, (1, 12)), (台灣咖啡考察團, ORG, (...","[(由, P), (外交部, Nc), (中美洲, Nc), (經貿, Na), (辦事處,...","[(咖啡, 28), (台灣, 18), (瓜地馬拉, 9), (瓜國, 6), (中美洲,..."


In [43]:
df.iloc[0].top_key_freq

[('台灣', 20),
 ('教友', 7),
 ('札尼', 4),
 ('布雷西亞', 4),
 ('李世明', 4),
 ('教廷', 3),
 ('扶輪社', 3),
 ('梵蒂岡', 2),
 ('宗座', 2),
 ('圖書館', 2),
 ('北義', 2),
 ('國家', 2),
 ('羅馬', 2),
 ('使館', 2),
 ('總統', 2),
 ('重要性', 2),
 ('全球', 2),
 ('中國', 2),
 ('人民', 2),
 ('希望', 2)]

# Abstract (summary) and sentimental score(摘要與情緒分數)

Currently we don't use abstract in our later application. However, we prepare news summary for future usage.

We use snowNLP to get summary and sentimental score.

In [44]:
# pip install snownlp

In [45]:
%%time
summary=[]
sentiment=[]
for text in df.content: # process piece by piece
    summary.append("暫無")  
    sentiment.append("暫無")

CPU times: total: 0 ns
Wall time: 0 ns


In [46]:
df['summary'] = summary

In [47]:
df['sentiment'] = sentiment

In [48]:
df.head(1)

,item_id,date,category,title,content,link,photo_link,tokens,tokens_v2,entities,token_pos,top_key_freq,summary,sentiment
0,aipl_20250304_1,2025-03-04,政治,梵蒂岡圖書館長率團參觀台灣使館 力薦值得訪台,梵蒂岡宗座圖書館總館長札尼今天率領數十位北義的社會菁英教友拜會駐教廷大使館。曾訪台的札尼致詞...,/news/aipl/202503040022.aspx,https://imgcdn.cna.com.tw/www/WebPhotos/200/20...,"[梵蒂岡, 宗座, 圖書館, 總, 館長, 札尼, 今天, 率領, 數十, 位, 北義, 的...","[梵蒂岡, 宗座, 圖書館, 館長, 札尼, 北義, 社會, 菁英, 教友, 教廷, 大使館...","[(梵蒂岡, GPE, (0, 3)), (札尼, PERSON, (11, 13)), (...","[(梵蒂岡, Nc), (宗座, Na), (圖書館, Nc), (總, Nc), (館長,...","[(台灣, 20), (教友, 7), (札尼, 4), (布雷西亞, 4), (李世明, ...",暫無,暫無


# Rearrange the colmun order for readability

In [49]:
df.columns

Index(['item_id', 'date', 'category', 'title', 'content', 'link', 'photo_link',
       'tokens', 'tokens_v2', 'entities', 'token_pos', 'top_key_freq',
       'summary', 'sentiment'],
      dtype='object')

In [50]:
# This operation is the same as slicing
df=df[['item_id', 'date', 'category', 'title', 'content','sentiment', 'summary', 'top_key_freq', 'tokens',
       'tokens_v2', 'entities', 'token_pos', 'link', 'photo_link'
       ]]

In [51]:
df.head(1)

,item_id,date,category,title,content,sentiment,summary,top_key_freq,tokens,tokens_v2,entities,token_pos,link,photo_link
0,aipl_20250304_1,2025-03-04,政治,梵蒂岡圖書館長率團參觀台灣使館 力薦值得訪台,梵蒂岡宗座圖書館總館長札尼今天率領數十位北義的社會菁英教友拜會駐教廷大使館。曾訪台的札尼致詞...,暫無,暫無,"[(台灣, 20), (教友, 7), (札尼, 4), (布雷西亞, 4), (李世明, ...","[梵蒂岡, 宗座, 圖書館, 總, 館長, 札尼, 今天, 率領, 數十, 位, 北義, 的...","[梵蒂岡, 宗座, 圖書館, 館長, 札尼, 北義, 社會, 菁英, 教友, 教廷, 大使館...","[(梵蒂岡, GPE, (0, 3)), (札尼, PERSON, (11, 13)), (...","[(梵蒂岡, Nc), (宗座, Na), (圖書館, Nc), (總, Nc), (館長,...",/news/aipl/202503040022.aspx,https://imgcdn.cna.com.tw/www/WebPhotos/200/20...


# Save data to disk

存檔

In [52]:
df.to_csv('cna_news_preprocessed.csv', sep='|', index=False)

## Read it out 讀出看看

In [53]:
df = pd.read_csv('cna_news_preprocessed.csv', sep='|')

In [54]:
df.head(1)

,item_id,date,category,title,content,sentiment,summary,top_key_freq,tokens,tokens_v2,entities,token_pos,link,photo_link
0,aipl_20250304_1,2025-03-04,政治,梵蒂岡圖書館長率團參觀台灣使館 力薦值得訪台,梵蒂岡宗座圖書館總館長札尼今天率領數十位北義的社會菁英教友拜會駐教廷大使館。曾訪台的札尼致詞...,暫無,暫無,"[('台灣', 20), ('教友', 7), ('札尼', 4), ('布雷西亞', 4)...","['梵蒂岡', '宗座', '圖書館', '總', '館長', '札尼', '今天', '率...","['梵蒂岡', '宗座', '圖書館', '館長', '札尼', '北義', '社會', '...","[NerToken(word='梵蒂岡', ner='GPE', idx=(0, 3)), ...","[('梵蒂岡', 'Nc'), ('宗座', 'Na'), ('圖書館', 'Nc'), (...",/news/aipl/202503040022.aspx,https://imgcdn.cna.com.tw/www/WebPhotos/200/20...
